导入相关的包
pip install pandas
pip install ddddocr
pip install selenium

In [1]:
import time
import ddddocr
from selenium.common import NoSuchElementException, InvalidSelectorException,ElementClickInterceptedException
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd


In [2]:
"""加载OCR，识别验证码用"""
ocr = ddddocr.DdddOcr(show_ad=False)

In [3]:
"""加载客户清单"""
customers = pd.read_excel("tmp/customers.xlsx", dtype=str)
searchResult = pd.DataFrame(columns=['身份证', '法院名称', '地区名称', '立案时间', '案号',
                                     '做出执行依据单位', '生效法律文书确定的义务',
                                     '被执行人的履行情况', '失信被执行人行为具体情形'])

with open("tmp/pos", "r") as f:
    pos = str(f.read())
if len(pos) == 0:
    pos = 2
else:
    pos = int(pos)
print("从第 %d 行开始查询：" % pos)
pos -= 2


从第 2813 行开始查询：


In [4]:
"""selenium 启用缓存"""
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'
# chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")


In [5]:
class AutoFetcher:
    def __init__(self, drv):
        self.driver = drv

    def openPage(self, url):
        self.driver.get(url)

    def inputCardNum(self, idcard):
        driver.find_element(By.NAME, "pCardNum").send_keys(idcard)

In [8]:
disruptCount = 0
logFile = open("tmp/log.log", "a")

with open("tmp/pos", "r") as f:
    pos = str(f.read())
if len(pos) == 0:
    pos = 2
else:
    pos = int(pos)
print("从第 %d 行开始查询：" % pos)
pos -= 2

driver = webdriver.Chrome(chrome_options=chrome_options)
auto = AutoFetcher(drv=driver)

for i in range(pos, customers.shape[0]):
    idcard = customers[customers.columns[0]][i]
    while True:
        try:
            driver.get("http://zxgk.court.gov.cn/shixin/")
            driver.find_element(By.NAME, "pCardNum").send_keys(idcard)
            
            driver.find_element(By.ID, "captchaImg").click()
            time.sleep(2)
            img = driver.find_element(By.ID, "captchaImg")

            with open('tmp/1.png', 'wb') as file:
                file.write(img.screenshot_as_png)

            with open('tmp/1.png', 'rb') as f:
                img_bytes = f.read()
            check_code = ocr.classification(img_bytes)
            driver.find_element(By.ID, "yzm").clear()
            driver.find_element(By.ID, "yzm").send_keys(check_code)
            # driver.find_element_by_id("yzm").send_keys("ABCD")
            time.sleep(1)

            yzmWrong = driver.find_elements(By.XPATH, "//div/strong[text()='验证码错误!']/..")
            yzmRight = driver.find_element(By.XPATH, "//div/strong[text()='验证码正确!']/..")


            if not yzmRight.get_attribute("class").__contains__("hide") and yzmWrong[1].get_attribute("class").__contains__("hide"):
               break
            else:
                # print("验证码错误！")
                time.sleep(5)
        except NoSuchElementException:
            # print("%s is ok" % idcard)
            logFile.write("%s is ok\n" % idcard)
            logFile.flush()
            time.sleep(60)

    try:
        driver.find_element(By.XPATH, '//button[normalize-space()="查询"]').click()
        time.sleep(2)
        resultList = driver.find_elements(By.XPATH, '//a[text()="查看"]')
        if len(resultList) > 0:
            # for result in resultList[0:0]:
            result = resultList[0]
            result.click()
            time.sleep(1)
            courtNameDetail = driver.find_element(By.ID, "courtNameDetail").text
            areaNameDetail = driver.find_element(By.ID, "areaNameDetail").text
            regDateDetail = driver.find_element(By.ID, "regDateDetail").text
            caseCodeDetail = driver.find_element(By.ID, "caseCodeDetail").text
            gistUnitDetail = driver.find_element(By.ID, "gistUnitDetail").text
            dutyDetail = driver.find_element(By.ID, "dutyDetail").text
            performanceDetail = driver.find_element(By.ID, "performanceDetail").text
            disruptTypeNameDetail = driver.find_element(By.ID, "disruptTypeNameDetail").text

            print(idcard, courtNameDetail, areaNameDetail, areaNameDetail, regDateDetail, caseCodeDetail,
                  gistUnitDetail, performanceDetail, disruptTypeNameDetail)
            logFile.write("%s, %s, %s, %s, %s, %s, %s, %s, %s" % (idcard, courtNameDetail, areaNameDetail, areaNameDetail, regDateDetail, caseCodeDetail,
                  gistUnitDetail, performanceDetail, disruptTypeNameDetail))
            logFile.flush()

            # driver.find_element(By.XPATH, '//button[normalize-space()="关闭"]').click()
            logFile.write(idcard +"\n")
            logFile.flush()

            searchResult.loc[disruptCount] = [idcard, courtNameDetail, areaNameDetail, regDateDetail,
                                              caseCodeDetail, gistUnitDetail, dutyDetail, performanceDetail, disruptTypeNameDetail]
            disruptCount += 1
        # else:
            # print("%s is ok" % idcard)
    except InvalidSelectorException:
        print("%s is ok" % idcard)
    except NoSuchElementException:
        print("%s is ok" % idcard)

    with open("tmp/pos", "w") as f:
        f.write(str(i+2))
        f.flush()

    time.sleep(9)

with open("tmp/pos", "w") as f:
    f.write("2")
    f.flush()
searchResult.to_excel("tmp/查询结果.xlsx", index=None)
driver.close()

从第 3361 行开始查询：


WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=103.0.5060.53)
Stacktrace:
Backtrace:
	Ordinal0 [0x010DD953+2414931]
	Ordinal0 [0x0106F5E1+1963489]
	Ordinal0 [0x00F5C6B8+837304]
	Ordinal0 [0x00F4E45E+779358]
	Ordinal0 [0x00F4E04F+778319]
	Ordinal0 [0x00F4D646+775750]
	Ordinal0 [0x00F4C565+771429]
	Ordinal0 [0x00F4CB68+772968]
	Ordinal0 [0x00F58B10+822032]
	Ordinal0 [0x00F4DE5D+777821]
	Ordinal0 [0x00F4EA3B+780859]
	Ordinal0 [0x00F4E06A+778346]
	Ordinal0 [0x00F4D646+775750]
	Ordinal0 [0x00F4C565+771429]
	Ordinal0 [0x00F4CA3D+772669]
	Ordinal0 [0x00F5DEA4+843428]
	Ordinal0 [0x00FB4EBD+1199805]
	Ordinal0 [0x00FA449C+1131676]
	Ordinal0 [0x00FB4812+1198098]
	Ordinal0 [0x00FA42B6+1131190]
	Ordinal0 [0x00F7E860+976992]
	Ordinal0 [0x00F7F756+980822]
	GetHandleVerifier [0x0134CC62+2510274]
	GetHandleVerifier [0x0133F760+2455744]
	GetHandleVerifier [0x0116EABA+551962]
	GetHandleVerifier [0x01

In [ ]:
从第 2813 行开始查询：
331021198708230617 玉环市人民法院 浙江 浙江 2022年02月15日 （2022）浙1021执419号 浙江省玉环市人民法院 全部未履行 违反财产报告制度

In [ ]:
12180